## Controlling a two-link robotic arm from Python using zqmRemoteApi

In [4]:
# Import libraries
import zmqRemoteApi
import numpy as np
from time import sleep
from pathlib import Path

In [16]:
# Create a client to get connected to the zmqremoteApi sernver from CoppeliaSim
client = zmqRemoteApi.RemoteAPIClient()

# Get the remote object 'sim'
sim = client.getObject('sim')


In [64]:
UR5_sim_obj_id = "/UR5"
base_id = sim.getObject(UR5_sim_obj_id)

# the last joint is the rotor for the end effector
# does not affect the final position
no_joints = 6
joint_sim_names = [None for _ in range(6)]

joint_sim_names[0] = f"{UR5_sim_obj_id}/joint"

for i in range(1,len(joint_sim_names)):
    link_joint_str = "/link/joint" * i
    joint_sim_names[i] = f"{UR5_sim_obj_id}{link_joint_str}"

connection_name = f"{UR5_sim_obj_id}/connection"

In [59]:
# Assert that names have id. Error is thrown if id doesnt exists
joint_id = [
    sim.getObject(name) for name in joint_sim_names
]
connection_id = sim.getObject(connection_name)

In [34]:
def reset_arm():
    """
    Resets the arm position to 0 deg
    """
    for j in joint_id:
        sim.setJointTargetPosition(j, np.deg2rad(0))

In [37]:
sim.startSimulation()
reset_arm()

In [69]:

angles = [90, -45, 80, -40, 100, 80]
angles = [np.deg2rad(a) for a in angles]
assert len(angles) == no_joints

for a, j in zip(angles, joint_id):
    sim.setJointTargetPosition(j, a)

print("Connection final position")
#print(sim.getObjectPosition(connection_id, sim.id))



Connection final position


In [70]:
sim.stopSimulation()

1